---
title: "My Document"
format: html
bibliography: references.bib
link-citations: true
---


# Retention Time Alignment

Due to systemic error, datasets of chromatogams of samples run under the same experimental conditions often will exhibit retention time shifting for the same compound peak. Inter-sample data analysis requires that features are aligned within the same vector. The magnitude and direction of run on run peak shift is unique for each peak for each run, within a distribution, creating a complex problem. A traditional approach is to reduce the dimensionality of the chromatograms through an aggregate measure such as peak area, discarding the time axis in favor of an element-wise ordering. This approach has two downsides - the first is the manual marking, grouping and ordering of peaks across the sampleset, which is subjective and often irreproducible [@zheng_2017], the second is the arbitrary loss of information fed to the statistical model, specifically the signal shapes [@nielsen_1998]. @bos-RecentApplicationsChemometrics-2020 lists correlation-optimized warping (COW), dynamic time warping (DTW), and correlation-optimized shifting (COSHIFT) as the most popular methods of alignment.

## DTW

Dynamic time warping is a method of aligning two time series, a reference series (to be aligned to) and a query series (to be aligned), originally developed in the context of speech recognition technology [@velichko_1970]. Alignment achieved by performing localizaed warping in the form of  stretching and compressing of the query series until a pre-defined level of alignment is reached, measured as the minimization of the distances between the two series. The distance between the two signals is measured as the sum of the distances between each series elementwise pairing [@giorgino_2009]. [@jiao_2015] has shown that DTW is an appropriate method of aligning organic sample chromatogram datasets. [@bork_2013] has discussed the importance of DTW for process monitoring. They describe how traditional DTW can alter y-axis values while aligning the x, however an extension to the method termed 'Derivative Dynamic Time Warping' [@keogh_2001] respects the shape of each series by observing their first derivatives, reducing unnecessary modifications.

### Outputting Aligned Tensors Through DTW

Tomasi discusses signal alignment through DTW in [@tomasi_2004, p. 7, sec. 2.3.3.] where they note that DTW does not itself produce aligned series of the same length, rather outputting shortened or lengthened series depending on the warping path taken. For stacking of sample signals into a tensor, the signals need to be the same length. This is not the intent of the design of the DTW algorithm, which rather is used to output the cost of aligning the series in the form of a distance metric. They do state that a desired synchronization can be achieved by either taking the mean value of intervals of stretching in the query (removing repeated time points in the warping), or an asymmetric warping algorithm which directly maps the query to the reference, but this can cause discontinuities in the warped signal, quoting @kassidas_1997.

## COW

@skov_2006 discussed the use of the COW algorithm for alignment of chromatographic data. The Correlation Optimized Warping (COW) algorithm was developed by Nielsen et al. [@nielsen_1998] for the purpose of data prepraration for multivariate statistical analysis.

COW can be used on 2d and 3d data and the output can be fed directly into models such as PCA.[@nielsen_1998].

COW is similar to DTW constrained to a number of windows along the time axis [@nielsen_1998].

What is COW?

COW aligns one signal onto another through localized linear stretching and compression of its time axis.
COW was developed by Nielsen et al. who demonstrated its use on single and multi-channel HPLC-DAD chromatograms of fungal extracts [@nielsen_1998].

The theory of COW is as follows (using notation from [@nielsen_1998]): for two signals, the 'target' (T) and the 'profile' (P), the two signals are divided up into a finite number of sections ($N=\frac{L_p}{m}$), each of which is internally warped to maximise alignment, resulting in an aligned signal (P'). This operation is constrained to ensure that time ordering is retained. Within each section, warping can either stretch or compress the sections, and in the case of a length mismatch, P' is linearly interpolated to match the length of T. The warping magnitude is constrained through parameter $t$, called 'the slack'. For P and T of different length, slack is defined as lying within the range $\Delta \pm t$ where $\Delta=\frac{L_T}{N}-m$.  Warping is performed section by section, and for the correlation coefficient of each pair is calculated and the end-point of the process is the maximisation of the sum of correlation coefficients. The identification of the section warping optimization is identified through dynamic programming. The calculation only requires specification of the segment length and slack parameters. Nielsen et al. demonstrated that COW performed better when fitting 3D data than 2D as the 'spectral information' restricted overfitting. In their example, they showed that subsetting the dataset to the target interval and baseline correction improved alignment. They also recommended using the cubed correlation coefficient over the base form as it selects for optimzied alignment without compromise. [@nielsen_1998].

DTW and COW were compared by Tomasi et al. [@tomasi_2004]. The MATLAB code the group wrote for COW can be found on the website of [Chemometrics Group of Copenhagen](https://ucphchemometrics.com/warping/).

COW has been adapted for 2D chromtaography by several indepedant research groups [@zhang_2008, @gros_2012].


## DTW Algorithm

notes from @giorgino_ComputingVisualizingDynamic_2009.

### Introduction

$X$: the *test* or *query*
$Y$: the *reference*

*i*: index of $X$
*j*: index of $Y$

*f*: *local dissimilarity function* defined between pairs of $x_i$ and $y_j$. Non negative: $$d(i, j)=f(x_i, y_j) \geq 0$$

$d$: cross-distance matrix between $X$ and $Y$

$\phi(k)$: warping curve $$\phi(k)=(\phi_x(k), \phi_y(k))$$

Where $\phi_x(k), \phi_y(k)$ outputs an integer from 1 to N.

$\phi_x$ remaps $X$ time indices
$\phi_y$ remaps $Y$ time indices

There is a average accumulated distortion between the warped $X$ and $Y$: $$d_\phi(X, Y) = \sum_{k=1}^{T} d(\phi_x(k), \phi_y(k)) \frac{m_\phi(k)}{M_\phi}$$

$m_\phi(k)$: per-step weighting coefficient, 
$M_\phi(k)$: normalizing constant of $m_\phi(k)$

 $\phi$ is constrained to ensure reasonable results. One constraint is monotonicity to ensure time ordering/avoid unnecessary loops: $$\phi_x(k+1) \geq \phi_x(k)$$  $$\phi_y(k+1) \geq \phi_y(k)$$

The goal of DTW is to minimize the distance between $X$ and $Y$: $$D(X, Y)=min \space d_\phi(X, Y)$$

Note: this mentions that Y is also deformed "The deformation of the time axes of $X$ **and** $Y$" (emphasis mine).

DTW can be computed in $O(N \cdot M)$ tiime.

$D(X, Y)$: "minimum global dissimilarity", "DTW distance". Stretch insensitive measure of the 'inherent difference' between $X$ and $Y$


## dtwalign

`dtwalign` is a Python package that includes outputting the alignment path


In [ ]:
%load_ext autoreload
%autoreload 2

from pybaselines import Baseline
from dtwalign import dtw
import pandas as pd
import numpy as np
from wine_analysis_hplc_uv import definitions
import seaborn as sns
import matplotlib.pyplot as plt
from wine_analysis_hplc_uv.notebooks.dtw_methods import DTWNotebookMethods
from wine_analysis_hplc_uv.signal_processing.mindex_signal_processing import (
    SignalProcessor,
)

scipro = SignalProcessor()

nb_mtds = DTWNotebookMethods()

df = pd.read_parquet(definitions.XPRO_YPRO_DOWNSAMPLED_PARQ_PATH)
df.head()

To develop an optimal alignment, we will focus on the alignment of 2021 John Duval Shiraz with 2021 Torbreck Struie. 


In [ ]:
# join x, y and aligned x on index

x = df.loc[:, ["176"]].pipe(
    lambda df: df.set_axis(
        axis=1,
        labels=pd.MultiIndex.from_arrays(
            [["176"], ["query"], ["NA"], ["abs"]],
            names=["sample", "status", "window_size", "unit"],
        ),
    )
)

y = df.loc[:, ["177"]].pipe(
    lambda df: df.set_axis(
        axis=1,
        labels=pd.MultiIndex.from_arrays(
            [["177"], ["ref"], ["NA"], ["abs"]],
            names=["sample", "status", "window_size", "unit"],
        ),
    )
)

In [ ]:
# align 176 on 177 without any constraints

align_x = nb_mtds.dtw_align_series(x, y)

align_x.plot()

As we can see from the alignment plane plot and series overlays, a significant section of the query series has been compressed and then interpolated as a flat line, losing a number of peaks in the process, see below:


In [ ]:
plotdata, g = nb_mtds.query_ref_align_plot(x, y, align_x)

As we can see, numerous peaks are lost post-alignment, which is unacceptable.


## Windowing

When aligning a query and reference chromatogram of similar samples we have the expectation that peaks within $d_m$ distance, where $d_m$ is misalignment distance, are the same compound and will be aligned to the same retention time post warping. Conversely, we do not expect regions where the query has peaks but the reference does not to be altered. Unfortunatey, the algorithm does not by default contain that information, and without a global constraint it will drastically alter the query along the entire serie to minimize what it percieves as the distance between the two, including compressing peaks present in the query but not the reference:


As we cam see the query is modified by the algorithm to match the reference in both the x and y axes. This leads to an unexpectedly deformed query. What is needed is to restrict warping to localized regions in the form of a windowing operation with windows of specific geometry. One such is the Sakoe-Chiba band [@sakoe_1978] which restricts how far apart two elements can be when matched: $$|\phi_x(k)-\phi_y(k)| \leq T_0$$ where $T_0$ is the absolute time deviation between two matched elements, specified by the user. As described by @giorgino_ComputingVisualizingDynamic_2009, this creates a boundary within the alignment plane within which the warping path can exist. For a window of size 10:


In [ ]:
# get the windowed aligned x, assign a regular frequency time index, rename axes


sakoechiba_10_x_align = nb_mtds.dtw_align_series(
    x, y, dict(window_type="sakoechiba", window_size=10)
)
sakoechiba_10_x_align.plot()

Looks promising, how does it compare to the query and the reference?


In [ ]:
plotdata, __ = nb_mtds.query_ref_align_plot(x=x, y=y, x_align=sakoechiba_10_x_align)

In [ ]:
(
    plotdata.loc[:, pd.IndexSlice[:, :, :, :, "query and aligned query"]]
    .melt(ignore_index=False, value_name="mAU")
    .pipe(lambda df: df.set_index(keys=df.index.total_seconds() / 60))
    .pipe(sns.lineplot, x="mins", y="mAU", hue="status")
    .set_title("query and aligned query")
)

Success! Selecting a Sakoechiba window with size 10 dramaticaly alters the behavior of the warping, preserving peaks that were otherwise lost. It also appears as though the (in the default setting at least), baseline height differences affect the warp. It appears that reducing the distance/cost pre-warp will reduce overall warping errors. Before we continue we should examine the effects of baseline subtraction.


## DTW with Subtracted Baselines


For the same x and y, assess effect of DTW on baseline subtracted series


In [ ]:
data = pd.concat([x, y], axis=1)
data

In [ ]:
data = data.rename({"abs": "value"}, axis=1).pipe(
    lambda df: df.set_axis(df.columns.set_names("signal", level="unit"), axis=1)
)
data

A lam of 1000 has been chosen as that appears to fit the baseline to the base of each peak without fitting to the internal area of the peak


In [ ]:
# assign the baseline

from pybaselines import Baseline


def assign_baseline_correction(df) -> pd.DataFrame:
    df = data.pipe(
        lambda df: df.melt(ignore_index=False, value_name="raw")
        .drop("signal", axis=1)
        .pipe(
            lambda df: df.groupby("sample", group_keys=False).apply(
                lambda grp: grp.assign(
                    bline=Baseline(grp.index.total_seconds()).asls(grp.raw, lam=1000)[0]
                )
            )
        )
        .pipe(
            lambda df: df.groupby("sample", group_keys=False).apply(
                lambda grp: grp.assign(bcorr=grp.raw - grp.bline)
            )
        )
        .pivot(
            columns=["sample", "status", "window_size"],
            values=["raw", "bline", "bcorr"],
        )
        .pipe(lambda df: df.set_axis(df.columns.set_names("signal", level=0), axis=1))
        .reorder_levels(order=["sample", "status", "window_size", "signal"], axis=1)
        .sort_index(axis=1)
    )
    (
        df.melt(ignore_index=False)
        .pipe(sns.FacetGrid, col="sample")
        .map_dataframe(sns.lineplot, x="mins", y="value", hue="signal")
        .add_legend()
    )
    return df


data = assign_baseline_correction(data)

Now lets observe how the warping behaves for the same parameters


In [ ]:
dtw_align_series = nb_mtds.dtw_align_series(
    x=data.loc[:, pd.IndexSlice["176", :, :, "bcorr"]],
    y=data.loc[:, pd.IndexSlice["177", :, :, "bcorr"]],
)
data = (
    pd.concat([data.loc[:, pd.IndexSlice[:, :, :, "bcorr"]], dtw_align_series], axis=1)
    .sort_index(axis=1)
    .droplevel(axis=1, level=3)
)

In [ ]:
idx = pd.IndexSlice


def plot_bcorr_dtw(data):
    x = data.loc[:, idx[:, "query", :]]
    y = data.loc[:, idx[:, "ref", :]]
    x_align = data.loc[:, idx[:, "aligned", :]]

    sp1 = data.loc[:, pd.IndexSlice[:, ["query", "ref"]]].pipe(
        lambda df: df.set_axis(
            axis=1,
            labels=pd.MultiIndex.from_frame(
                df.columns.to_frame().assign(subplot="query and reference")
            ),
        )
    )

    # sp2 is aligned and ref

    sp2 = data.loc[:, pd.IndexSlice[:, ["aligned", "ref"]]].pipe(
        lambda df: df.set_axis(
            axis=1,
            labels=pd.MultiIndex.from_frame(
                df.columns.to_frame().assign(subplot="aligned query and reference")
            ),
        )
    )

    ## sp3 is query and aligned

    sp3 = data.loc[:, pd.IndexSlice[:, ["query", "aligned"]]].pipe(
        lambda df: df.set_axis(
            axis=1,
            labels=pd.MultiIndex.from_frame(
                df.columns.to_frame().assign(subplot="query and aligned query")
            ),
        )
    )

    # concatenate the three subplot dataframes, melt, renaming the value column to 'mAU', convert the date time index to a minutes float, create a column-wise facetgrid on 'subplot', map a lineplot to the facetgrids of 'mins', 'mAU', 'status' for hue, set the subplot titles to subplot value.

    plotdata = pd.concat([sp1, sp2, sp3], axis=1)

    g = (
        plotdata.melt(ignore_index=False, value_name="mAU")
        .pipe(lambda df: df.set_index(df.index.total_seconds() / 60))
        .pipe(
            lambda df: sns.FacetGrid(df, col="subplot")
            .map_dataframe(sns.lineplot, x="mins", y="mAU", hue="status")
            .set_titles(col_template="{col_name}")
            .add_legend()
        )
    )


plot_bcorr_dtw(data)

I would call that successful without any further modifications to the dtw algorithm. At this point in time we need a number of tools -

1. a method of evaluating alignment
2. produce a matrix of subplots for each sample in the set row wise.


In [ ]:
# find a way to construct the subplot dataset through groupby on sample

In [ ]:
# generate a 2x2 plot grid of each sample with an overlay of the raw, the baseline and
# the bcorr signals

df = (
    df.melt(ignore_index=False, value_name="raw")
    .groupby("samplecode", group_keys=False)
    .apply(
        lambda grp: grp.assign(
            bline=Baseline(grp.index.total_seconds()).asls(grp.raw, lam=1000)[0]
        )
    )
    .assign(bcorr=lambda df: df.raw - df.bline)
    .pivot(columns=["samplecode", "wine"], values=["raw", "bline", "bcorr"])
    .pipe(lambda df: df.set_axis(df.columns.set_names(level=0, names="signal"), axis=1))
    .reorder_levels(axis=1, order=["samplecode", "wine", "signal"])
    .sort_index(axis=1)
)

(
    df.melt(ignore_index=False, value_name="mAU")
    #  .pipe(lambda df: df.set_index())
    .pipe(sns.FacetGrid, col="wine", col_wrap=2)
    .map_dataframe(sns.lineplot, hue="signal", x="mins", y="mAU")
    .set_titles(col_template="{col_name}")
    .add_legend()
)

The baseline corrected signals appear to be acceptable to me. Does it modify which is the reference?


In [ ]:
ref = scipro.most_correlated(df)
ref

It does, 176 is now the most correlated.


In [ ]:
y = df.loc[:, idx[ref, :, "bcorr"]]

# add aligned series to original df through concatenation. Need to subset the original
# with the warping path, reindex to 2S timedelta range, rename signal level to 'aligned
# then concatentate with original df

df = (
    df.loc[:, idx[:, :, "bcorr"]]
    .pipe(
        lambda df: df.groupby(["wine"], group_keys=False, axis=1).apply(
            lambda df: pd.concat(
                [
                    df,
                    df.iloc[dtw(x=df, y=y).get_warping_path(), :].pipe(
                        lambda df: df.set_index(
                            pd.timedelta_range(
                                start=df.index[0], end=df.index[-1], freq="2S"
                            )
                        )
                        .rename_axis("mins")
                        .rename({"bcorr": "aligned"}, axis=1)
                    ),
                ],
                axis=1,
            )
        )
    )
    .pipe(
        lambda df: df.reindex(
            axis=1,
            labels=pd.MultiIndex.from_frame(
                df.columns.to_frame()
                .assign(role="query")
                .assign(
                    role=lambda df: df.loc[:, "role"].where(
                        ~(df.samplecode.isin(ref)), "ref"
                    )
                )
            ),
        )
    )
    .reorder_levels(axis=1, order=["samplecode", "wine", "role", "signal"])
    .sort_index(axis=1)
)
df

In [ ]:
# now we want to form the sets of three again.
"""
we are expecting to create a column 'subplot' with a pattern of 1,2,3 and for each
sample a combination of ('query','bcorr'), ('query','aligned'), and ('ref','bcorr').

specifically,

1: ('query','bcorr'), ('ref','bcorr')
2: ('query','aligned'), ('ref','bcorr')
3: ('query','bcorr'), ('query','aligned')

For each we're expecting 2 rows per sample, 8 rows for subplot 1, 8 rows for subplot 2, 8 rows for subplot 3.
We need combinations with repeats using the cartesian product (?)
"""


# get the reference



new_index_ = (df
 .columns
 .to_frame(index=False)
 .assign(state=lambda df: df
         .signal
         .where(~(df.signal=='aligned'),'aligned')
         .where(~(df.signal=='bcorr'),'query')
         .where(~(df.role=='ref'),'ref')
         )
 .drop(['role','signal'],axis=1)
 )

In [ ]:
new_index=pd.MultiIndex.from_frame(new_index_)
df = df.set_axis(new_index, axis=1)

In [ ]:


a = df.columns.to_frame(index=False).set_index(["samplecode", "wine", "state"]).loc[~a.index.duplicated(keep='first'),:]
display(a)

In [ ]:
ref_idx = a.loc[idx[:, :, 'ref']].assign(subplot="1").index

In [ ]:
"""         
| sample  | row  | col  | role  |
|---------|------|------|-------|
| sample1 | row1 | col1 | query |
| sample1 | row1 | col1 | ref   |
| sample1 | row1 | col2 | align |
| sample1 | row1 | col2 | ref   |
| sample1 | row1 | col3 | query |
| sample1 | row1 | col3 | align |

do for each, then concat horizontally.

The iteration pattern is:

sample 1, 2, 3
row 1, 2, 3
column 1, 2, 3

for sample in samples:
    for column in range(0:2):
        col1 = 
"""

samples = a.index.get_level_values("samplecode").unique()


# get sample row
def build_col1(df, sample_id):
    """
    combine query and reference for overlaying in col1
    """
    
    if sample_id=='176':
        sample = df.loc[idx[sample_id, :, "ref"], :]
    
    else:
        sample = df.loc[idx[sample_id, :, "query"], :]


    # get the reference row, reindex it so its 'wine' (row) is s1
    
    if sample_id=='176':
        ref=df.loc[idx[:,:,'ref'],:]
    
    else:
        wine = sample.index.get_level_values("wine")
        ref = df.loc[idx[:, :, "ref"], :].pipe(
            lambda df: df.set_axis(
                df.index.remove_unused_levels().set_levels(level=["wine"], levels=[wine])
            )
        )

    # assign row and column identifier to reference and sample rows
    col1 = pd.concat([sample, ref]).assign(col=1)

    return col1


def build_col2(df, sample_id):
    """
    combine aligned and reference for overlaying in col1
    """
    if sample_id=='176':
        sample = df.loc[idx[sample_id, :, "ref"], :]
    
    else:
        sample = df.loc[idx[sample_id, :, "aligned"], :]


    # get the reference row, reindex it so its 'wine' (row) is s1
    
    if sample_id=='176':
        ref=sample.copy()
    
    else:
        wine = sample.index.get_level_values("wine")
        ref = df.loc[idx[:, :, "ref"], :].pipe(
            lambda df: df.set_axis(
                df.index.remove_unused_levels().set_levels(level=["wine"], levels=[wine])
            )
        )

    # assign row and column identifier to reference and sample rows
    
    col2 = pd.concat([sample, ref]).assign(col=2)
    
    return col2


def build_col3(df, samplecode):
    """
    combined query and aligned
    """

    if samplecode=='176':
        col3 = df.loc[[samplecode]].assign(col=3)
        
    else:
        col3 = df.loc[idx[samplecode, :, ["query", "aligned"]], :].assign(col=3)

    return col3


col1 = pd.concat([build_col1(a, sample) for sample in samples])
col1 = col1.assign(row=col1.groupby("wine").ngroup() + 1)
col2 = pd.concat([build_col2(a, sample) for sample in samples]).assign(
    row=lambda df: df.groupby("wine").ngroup() + 1
)
col3 = pd.concat([build_col3(a, sample) for sample in samples]).assign(
    row=lambda df: df.groupby("wine").ngroup() + 1
)

index_df = pd.concat([col1, col2, col3])
# index_df = index_df[["row", "col"]]



index_df.set_index(["row", "col"], append=True).reorder_levels(
    ["row", "col", "wine", "state","samplecode"]
).sort_index(level=["row", "col", "state"])

In [ ]:
# can we just reindex the original df?


def join_df_index_df(df, index_df):
    """
    massage df and index_df to left join onto index_df
    """

    pdf =(
        df
        .melt(ignore_index=False)
        .reset_index()
        .set_index(['samplecode','state'])
        .drop('wine',axis=1) 
    )

    pindex_df = index_df.reset_index().set_index(["samplecode",'state'])
    display('pindex_df')
    display(pindex_df)
    display('pdf')
    display(pdf)
    join = pindex_df.join(pdf, how='left').dropna()

    return join


relplotdata = join_df_index_df(df, index_df)
display(relplotdata)
display(relplotdata.describe())
display(relplotdata.info())

In [ ]:
# convert timedelta to float

relplotdata = relplotdata.assign(mins=lambda df: df.mins.dt.total_seconds() / 60)

In [ ]:
df_means = df.mean().to_frame("mean").reset_index().set_index("mean")
df_means

In [ ]:
# check if joins acted as expected by calulating an aggregation statisic, say mean.


post_join_means = (
    relplotdata.groupby(["samplecode", "wine", "state", "row", "col"])["value"]
    .mean()
    .to_frame(name="mean")
)

post_join_means = (
    post_join_means.reorder_levels(["row", "col", "samplecode", "wine", "state"])
    .reset_index()
    .set_index("mean")
    .
)
post_join_means

In [ ]:
df_means.loc[lambda df: ~df.index.duplicated(keep='first'),:]

In [ ]:
post_join_means.join(df_means, lsuffix="left", rsuffix="right")

In [ ]:
# pd.concat([df_means, post_join_means],axis=1)

In [ ]:
(
    relplotdata.set_index(["row", "col"], append=True)
    .loc[idx[:, 1, 3]]
    .pipe(lambda df: df if display(df) else df)  # display df
    #  .reset_index()
    .pipe(sns.relplot, x="mins", y="value", hue="state", legend="full", kind="line")
)

In [ ]:
# now display it..

sns.relplot(
    relplotdata,
    col="col",
    row="row",
    x="mins",
    y="value",
    hue="state",
    kind="line",
    legend="full",
)

In [ ]:
col1 = df.loc[idx[:, :, ["query", "ref"], "bcorr"], :].assign(col="col1")
display(col1)

col2 = (
    pd.concat(
        [
            df.loc[idx[:, :, "query", "aligned"], :],
            df.loc[idx[:, :, "ref", "bcorr"], :],
        ]
    )
    .sort_index()
    .assign(col="col2")
)

display(col2)

col3 = (
    pd.concat(
        [
            df.loc[idx[:, :, "query", "bcorr"], :],
            df.loc[idx[:, :, "query", "aligned"], :],
        ]
    )
    .sort_index()
    .assign(col="col3")
)

display(col3)

pdata = pd.concat([col1, col2, col3]).sort_index()
pdata

In [ ]:
# sp3 = a[(a['role']=='query') & (a['signal']=='bcorr') | (a['role']=='query') & (a['signal']=='aligned')].assign(subplot='3')

# display(sp3)

# subplots_index = pd.concat([sp1,sp2,sp3], ignore_index=True)

# subplots_index.set_index(['samplecode','wine','role','signal'])

In [ ]:
# use `relplot` rather than FacetGrid

In [ ]:
plot_df = df.reindex(pd.MultiIndex.from_frame(subplots_index), axis=1)
plot_df

In [ ]:
(
    plot_df.melt(ignore_index=False, value_name="mAU").pipe(
        lambda df: sns.FacetGrid(df, col="subplot", row="wine")
        .map_dataframe(sns.lineplot, x="mins", y="mAU", hue="role")
        .add_legend()
    )
)

In [ ]:
help(sns.relplot)

In [ ]:
help(df.columns)